In [1]:
import pandas as pd
import sqlite3

In [136]:
conn = sqlite3.connect("pets.db")

### You want to execute commands through cursors, so we will create a cursor for our connection

In [137]:
cur = conn.cursor()

If you want to close your connection

In [138]:
conn.close()

Then you have to reconnect

In [29]:
create_table_query = """
CREATE TABLE IF NOT EXISTS pets (
    name text,
    species text,
    color text,
    age int,
    human text
);
"""
cur.execute(create_table_query)

In [30]:
conn.commit()

In [31]:
insert_query = """
INSERT INTO pets
(name, species, color, age, human)
VALUES ("Chloe", "dog", "black", 9, "Tim");
"""

In [45]:
cur = conn.cursor()
cur.execute(insert_query)
conn.commit()

### Create a custom function to add pets

In [38]:
def add_pet(name, species, color, age, human, conn):
    cur = conn.cursor()
    query = f"""
    INSERT INTO pets
    (name, species, color, age, human)
    VALUES ("{name}", "{species}", "{color}", {age}, "{human}")
    """
    cur.execute(query)
    conn.commit()

In [47]:
add_pet("Muffy", 'mutt', 'brown', 7, 'Arman', conn)

### Create custom function to get items by name

In [40]:
def get_by_name(name, conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM pets WHERE name = ?", (name, ))
    data_raw = cur.fetchall()
    return pd.DataFrame(
        data_raw,
        columns=["name", "species", "color", "age", "human"]
    )

In [56]:
get_by_name("Chloe", conn)

,name,species,color,age,human
0,Chloe,dog,black,9,Tim
1,Chloe,dog,black,9,Tim
2,Chloe,dog,black,9,Tim
3,Chloe,dog,black,9,Tim
4,Chloe,dog,black,9,Tim
5,Chloe,dog,black,9,Tim


### Instead of a bunch of functions, let's put them all into one Class

In [2]:
class PetDB:
    def __init__(self, file):
        self.conn = sqlite3.connect(file)
        self.cur = self.conn.cursor()
        
    def create_table(self):
        create_table_query = """
            CREATE TABLE IF NOT EXISTS pets (
                name text,
                species text,
                color text,
                age int,
                human text
            )
        """

        self.cur.execute(create_table_query); 
        self.conn.commit()
        
    def drop_table(self):
        self.cur.execute("DROP TABLE IF EXISTS pets")
        self.conn.commit()

    def add_pet(self, name, species, color, age, human):
        query =  f"""
            INSERT INTO pets
            (name, species, color, age, human)
            VALUES ("{name}", "{species}", "{color}", {age}, "{human}")
        """
        
        self.cur.execute(query)
        self.conn.commit()
    
    def get_all(self):
        self.cur.execute("SELECT * FROM pets")
        return pd.DataFrame(
            self.cur.fetchall(),
            columns=["name", "species", "color", "age", "human"]
        )
        
    def get_by_name(self, name):
        self.cur.execute("SELECT * FROM pets WHERE name = ?", (name,))
        return pd.DataFrame(
            self.cur.fetchall(),
            columns=["name", "species", "color", "age", "human"]
        )
    
    def had_birthday(self, name):
        old_age = self.get_by_name(name).age[0]
        
        update_query = f"""
        UPDATE pets
        SET age = {old_age + 1}
        WHERE name = "{name}"
        """
        
        self.cur.execute(update_query)
        self.conn.commit()
        
    def delete_by_name(self, name):
        self.cur.execute("DELETE FROM pets WHERE name = ?", (name,))
        self.conn.commit()

In [3]:
db = PetDB("pets.db")

In [6]:
db.add_pet("Midge", "cat", "black orange", 7, "Brendan")

In [7]:
db.get_all()

,name,species,color,age,human
0,Muffy,mutt,brown,7,Arman
1,Muffy,mutt,brown,7,Arman
2,Muffy,mutt,brown,7,Arman
3,Muffy,mutt,brown,7,Arman
4,Midge,cat,black orange,7,Brendan
5,Midge,cat,black orange,7,Brendan
6,Midge,cat,black orange,7,Brendan


In [8]:
db.delete_by_name("Midge")

In [9]:
db.get_all()

,name,species,color,age,human
0,Muffy,mutt,brown,7,Arman
1,Muffy,mutt,brown,7,Arman
2,Muffy,mutt,brown,7,Arman
3,Muffy,mutt,brown,7,Arman


In [10]:
db.delete_by_name("Muffy")

In [11]:
db.get_all()

,name,species,color,age,human


In [12]:
db.add_pet("Midge", "cat", "black orange", 7, "Brendan")

In [14]:
db.add_pet("Muffy", 'dog', 'brown', 7, 'Arman')

In [15]:
db.get_all()

,name,species,color,age,human
0,Midge,cat,black orange,7,Brendan
1,Muffy,dog,brown,7,Arman


In [16]:
db.add_pet("Chloe", "dog", "black", 9, "Tim")

In [17]:
db.get_all()

,name,species,color,age,human
0,Midge,cat,black orange,7,Brendan
1,Muffy,dog,brown,7,Arman
2,Chloe,dog,black,9,Tim
